In [3]:
import os
import numpy as np
import pandas as pd
import psycopg2

In [10]:
ls

 Volume in drive D has no label.
 Volume Serial Number is 0C45-8BC1

 Directory of d:\Coding\Git\PosgresIngco

08/24/2022  03:48 PM    <DIR>          .
08/24/2022  03:48 PM    <DIR>          ..
08/24/2022  03:08 PM               917 pen jualan$.csv
08/19/2022  11:22 AM                34 README.md
08/24/2022  01:25 PM             1,728 test.ipynb
               3 File(s)          2,679 bytes
               2 Dir(s)  56,710,979,584 bytes free


In [9]:
df = pd.read_csv("pen jualan$.csv")
df.head()

,Tanggal,Sku,No. Pesanan,QTY,Asal Gudang,Total,Sumber,Jenis Penjualan,Kode Sumber
0,10/13/2020,CAGLI1001,INV/20200928/XX/IX/639426010,1,HQ [Online] - Bintaro,"Rp800,000",Tokopedia,Online,4
1,10/13/2020,HKISSD12101,INV/20201013/XX/X/650891804,1,TC Jakarta - TokoCabang,"Rp221,500",Tokopedia,Online,4
2,10/13/2020,ID6808,INV/20201013/XX/X/651141057,1,TC Jakarta - TokoCabang,"Rp384,413",Tokopedia,Online,4
3,10/14/2020,HKNS16518,INV/20201014/XX/X/651431074,1,TC Jakarta - TokoCabang,"Rp15,500",Tokopedia,Online,4
4,10/14/2020,HKSD0628,INV/20201014/XX/X/651431074,1,TC Jakarta - TokoCabang,"Rp71,500",Tokopedia,Online,4


In [15]:
#rename csv, force lower case, no spaces, no dashes
file = "pen jualan$"
clean_tbl_name = file.lower().replace(" ", "_").replace("-","_").replace(r"/","_").replace("\\","_").replace("$","").replace("%","")

clean_tbl_name

'pen_jualan'

In [19]:
conn_string = "host= zubdb.cabyoqiwyaa1.us-west-1.rds.amazonaws.com dbname= zubdb user= zubdev password= " 
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()
print('opened database successfully')

OperationalError: connection to server at "zubdb.cabyoqiwyaa1.us-west-1.rds.amazonaws.com" (52.53.110.229), port 5432 failed: FATAL:  database "zubdb" does not exist
